In [5]:
import pandas as pd
# Importa libreria geopandas como gpd
import geopandas as gpd

# 1. Extracción de las MANZANAS en Shapefile

In [2]:
# Lee archivo shapefile de distritos del norte de Irlanda.
# Antes hay que asegurarse de crear la carpeta Shapefiles y subir todos los archivos de Shapefiles.zip
gdf_districts = gpd.read_file('MANZANA.shp') # Distritos del norte de Irlanda

# muestra el tipo de dato
type(gdf_districts)

geopandas.geodataframe.GeoDataFrame

In [3]:
gdf_Lima = gdf_districts.copy()
gdf_Lima = gdf_Lima[(gdf_Lima.DEPARTAMEN == 'LIMA') & (gdf_Lima.PROVINCIA == 'LIMA')]

In [4]:
gdf_Lima.head(5)

,IDMANZANA,CODDPTO,CODPROV,CODDIST,CODZONA,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
340925,150101013000450,15,01,01,013,00,045,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,3.140281e-07,0.002454,"POLYGON ((-77.06513 -12.05393, -77.06549 -12.0..."
340926,150101013000410,15,01,01,013,00,041,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,4.670634e-07,0.003969,"POLYGON ((-77.06667 -12.0535, -77.0663 -12.053..."
340927,150101013000350,15,01,01,013,00,035,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,6.790691e-08,0.001773,"POLYGON ((-77.06712 -12.05334, -77.06711 -12.0..."
340928,150101013000370,15,01,01,013,00,037,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,1.303425e-07,0.002416,"POLYGON ((-77.06729 -12.05376, -77.06739 -12.0..."
340929,150101013000340,15,01,01,013,00,034,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,6.984726e-08,0.001533,"POLYGON ((-77.06751 -12.05394, -77.06758 -12.0..."


In [5]:
gdf_Lima.shape

(107100, 18)

# 2. Extracción de Albergues potenciales

## Centros recreacionales

In [6]:
df_recreacion = pd.read_excel('Equipamiento de recreación pública (PLANMET 2040).xlsx')

In [7]:
print('Registros de Centros recreacionales en BRUTO: ',df_recreacion.shape[0])

Registros de Centros recreacionales en BRUTO:  6655


In [8]:
df_recreacion.head(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,OBSERV,NOMBDIST,NOMBPROV,NOMBDEP,Shape__Area,Shape__Length
0,1,RE-1042,150139002000130,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Miami,8521.913596,CEMENTO,SANTA ROSA,LIMA,LIMA,8521.91,420.40
1,2,RE-1043,150139004000190,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA PRINCIPAL,Plaza de Armas de Santa Rosa,3521.627604,CEMENTO,SANTA ROSA,LIMA,LIMA,3521.63,257.03
2,3,RE-1044,150125010000490,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,2521.858226,VERDE,PUENTE PIEDRA,LIMA,LIMA,2521.86,397.11
3,4,RE-1045,15012504800011D,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Virgen de la Medalla Milagrosa,5417.728509,VERDE,PUENTE PIEDRA,LIMA,LIMA,5417.73,329.34
4,5,RE-1046,150125002000480,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Plaza Jerusalén,4727.502509,VERDE,PUENTE PIEDRA,LIMA,LIMA,4727.50,275.62


In [9]:
df_recreacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6655 entries, 0 to 6654
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OBJECTID       6655 non-null   int64  
 1   COD_EQ         6654 non-null   object 
 2   ID_MANZANA     5530 non-null   object 
 3   IDLOTE         0 non-null      float64
 4   IDMZNAR        169 non-null    object 
 5   COD_SECT       0 non-null      float64
 6   TIPO_EQ        6654 non-null   object 
 7   SUBTIPO_EQ     6655 non-null   object 
 8   NOM_EQ         3019 non-null   object 
 9   AREA_M2        6655 non-null   float64
 10  OBSERV         6654 non-null   object 
 11  NOMBDIST       6655 non-null   object 
 12  NOMBPROV       6654 non-null   object 
 13  NOMBDEP        6654 non-null   object 
 14  Shape__Area    6655 non-null   float64
 15  Shape__Length  6655 non-null   float64
dtypes: float64(5), int64(1), object(10)
memory usage: 832.0+ KB


In [10]:
df_recreacion = df_recreacion.rename(columns={
    'Shape__Area': 'Shape_Area',
    'Shape__Length': 'Shape_Length'
})

In [11]:
# Convertir la columna ID_MANZANA a string
df_recreacion['ID_MANZANA'] = df_recreacion['ID_MANZANA'].astype(str)

In [12]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre gdf_Lima y df_recreacion_filter
# La clave de unión es "ID_MANZANA"
gdf_recreacion = df_recreacion.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='inner')

In [13]:
print('Centros recreacionales que tienen Shapefile: ', gdf_recreacion.shape[0])

Centros recreacionales que tienen Shapefile:  5471


In [14]:
gdf_recreacion.sample(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
303,348,RE-1257,15010602301013B,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,1327.273866,...,150106,0001,LIMA,LIMA,CARABAYLLO,CARABAYLLO,1501060001,9.721577e-08,0.001264,"POLYGON ((-77.03482 -11.85889, -77.03495 -11.8..."
3745,4268,RE-5610,150121005000570,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Parque Jose de la Puente Olavegoya,3257.067053,...,150121,0001,LIMA,LIMA,PUEBLO LIBRE,PUEBLO LIBRE,1501210001,2.729317e-07,0.002092,"POLYGON ((-77.06469 -12.07987, -77.06472 -12.0..."
5153,6076,RE-6432,150122001000400,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Parque Federico Villarreal,706.133547,...,150122,0001,LIMA,LIMA,MIRAFLORES,MIRAFLORES,1501220001,7.538273e-08,0.001190,"POLYGON ((-77.04869 -12.11318, -77.04866 -12.1..."
4739,5487,RE-0809,15014202100058A,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,3349.867213,...,150142,0001,LIMA,LIMA,VILLA EL SALVADOR,VILLA EL SALVADOR,1501420001,4.011385e-07,0.002535,"POLYGON ((-76.95757 -12.21052, -76.95705 -12.2..."
2140,2375,RE-3649,150132120000500,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,938.660420,...,150132,0001,LIMA,LIMA,SAN JUAN DE LURIGANCHO,SAN JUAN DE LURIGANCHO,1501320001,7.789373e-08,0.001413,"POLYGON ((-77.01674 -12.02817, -77.01665 -12.0..."


In [15]:
# DATOS QUE NO SE CRUZAN
# Realizar el outer join entre gdf_Lima y df_recreacion_filter
gdf_result = df_recreacion.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='outer', indicator=True)

# Filtrar los registros que no encontraron coincidencia
no_match_recreacion = gdf_result[gdf_result['_merge'] == 'left_only']

In [16]:
no_match_recreacion_rastreable = no_match_recreacion.dropna(subset=['NOM_EQ'])

In [17]:
print('Centros recreacionales que NO tienen Shapefile, pero son rastreables: ', no_match_recreacion_rastreable.shape[0])

Centros recreacionales que NO tienen Shapefile, pero son rastreables:  234


In [18]:
no_match_recreacion_rastreable.head(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry,_merge
7,8.0,RE-1049,nan,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Parque Miraflores,1596.316754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
9,42.0,RE-4862,nan,NaN,NaN,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela Inca Garcilaso de la Vega,179.870590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
10,64.0,RE-4868,nan,NaN,MZ-0000007,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela Jardin de Santa Rosa,1613.930194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
11,68.0,RE-4872,nan,NaN,NaN,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela San Agustín,648.665773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
12,71.0,RE-4875,nan,NaN,NaN,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela Las Limeñitas,966.531211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only


## Centros deportivos

In [19]:
df_deportivo = pd.read_excel('Equipamiento deportivo (PLANMET 2040).xlsx')

In [20]:
df_deportivo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4216 entries, 0 to 4215
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OBJECTID      4216 non-null   int64  
 1   COD_EQ        4216 non-null   object 
 2   ID_MANZANA    3939 non-null   object 
 3   IDLOTE        0 non-null      float64
 4   IDMZNAR       0 non-null      float64
 5   COD_SECT      0 non-null      float64
 6   TIPO_EQ       4216 non-null   object 
 7   SUBTIPO_EQ    4216 non-null   object 
 8   NOM_EQ        370 non-null    object 
 9   AREA_M2       4216 non-null   float64
 10  OBSERV        0 non-null      float64
 11  NOMBDIST      4216 non-null   object 
 12  NOMBPROV      4216 non-null   object 
 13  NOMBDEP       4216 non-null   object 
 14  NOM_UA        0 non-null      float64
 15  Shape_Area    4216 non-null   float64
 16  Shape_Length  4216 non-null   float64
dtypes: float64(8), int64(1), object(8)
memory usage: 560.1+ KB


In [21]:
# Eliminar la columna NOM_UA del DataFrame
df_deportivo = df_deportivo.drop(columns=['NOM_UA'])

In [22]:
# Convertir la columna ID_MANZANA a string
df_deportivo['ID_MANZANA'] = df_deportivo['ID_MANZANA'].astype(str)

In [23]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre gdf_Lima y df_recreacion_filter
# La clave de unión es "ID_MANZANA"
gdf_deportivo = df_deportivo.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='inner')

In [24]:
print('Centros deportivos que tienen Shapefile: ', gdf_deportivo.shape[0])

Centros deportivos que tienen Shapefile:  3939


In [25]:
gdf_deportivo.head(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
0,3,D-2357,150132092000330,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,585.850150,...,150132,0001,LIMA,LIMA,SAN JUAN DE LURIGANCHO,SAN JUAN DE LURIGANCHO,1501320001,4.362677e-07,0.003102,"POLYGON ((-77.01442 -11.99518, -77.01446 -11.9..."
1,7,D-1026,150112012000420,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,1072.399392,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."
2,8,D-1027,150112012000420,NaN,NaN,NaN,OU:DEPORTE,CAMPO DEPORTIVO,EL NARANJAL,8491.443048,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."
3,9,D-1028,150112012000420,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,555.717492,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."
4,615,D-1104,150112012000420,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,475.503729,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."


In [26]:
# DATOS QUE NO SE CRUZAN
# Realizar el outer join entre gdf_Lima y df_recreacion_filter
gdf_result2 = df_deportivo.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='outer', indicator=True)

# Filtrar los registros que no encontraron coincidencia
no_match_deportivo = gdf_result2[gdf_result2['_merge'] == 'left_only']

In [27]:
no_match_deportivo_rastreable = no_match_deportivo.dropna(subset=['NOM_EQ'])

In [28]:
print('Centros deportivos que NO tienen Shapefile, pero son rastreables: ', no_match_deportivo_rastreable.shape[0])

Centros deportivos que NO tienen Shapefile, pero son rastreables:  12


# 3. Agrupamiento de Albergues potenciales: Recreativos + deportivos

In [29]:
# Concatenar los dos DataFrames a nivel de filas
gdf_combined = pd.concat([gdf_recreacion, gdf_deportivo], axis=0, ignore_index=True)

In [30]:
gdf_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9410 entries, 0 to 9409
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   OBJECTID      9410 non-null   int64   
 1   COD_EQ        9410 non-null   object  
 2   ID_MANZANA    9410 non-null   object  
 3   IDLOTE        0 non-null      float64 
 4   IDMZNAR       0 non-null      object  
 5   COD_SECT      0 non-null      float64 
 6   TIPO_EQ       9410 non-null   object  
 7   SUBTIPO_EQ    9410 non-null   object  
 8   NOM_EQ        3143 non-null   object  
 9   AREA_M2       9410 non-null   float64 
 10  OBSERV        5471 non-null   object  
 11  NOMBDIST      9410 non-null   object  
 12  NOMBPROV      9410 non-null   object  
 13  NOMBDEP       9410 non-null   object  
 14  Shape_Area    9410 non-null   float64 
 15  Shape_Length  9410 non-null   float64 
 16  IDMANZANA     9410 non-null   object  
 17  CODDPTO       9410 non-null   object  
 18  CODPROV 

In [31]:
gdf_combined

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
0,1,RE-1042,150139002000130,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Miami,8521.913596,...,150139,0001,LIMA,LIMA,SANTA ROSA,SANTA ROSA,1501390001,6.997925e-07,0.003825,"POLYGON ((-77.17481 -11.79968, -77.17481 -11.7..."
1,2,RE-1043,150139004000190,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA PRINCIPAL,Plaza de Armas de Santa Rosa,3521.627604,...,150139,0001,LIMA,LIMA,SANTA ROSA,SANTA ROSA,1501390001,4.455260e-07,0.003180,"POLYGON ((-77.15797 -11.78584, -77.15771 -11.7..."
2,3,RE-1044,150125010000490,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,2521.858226,...,150125,0001,LIMA,LIMA,PUENTE PIEDRA,PUENTE PIEDRA,1501250001,3.431174e-07,0.003262,"POLYGON ((-77.11043 -11.83321, -77.11045 -11.8..."
3,4,RE-1045,15012504800011D,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Virgen de la Medalla Milagrosa,5417.728509,...,150125,0001,LIMA,LIMA,PUENTE PIEDRA,PUENTE PIEDRA,1501250001,5.113287e-07,0.002939,"POLYGON ((-77.08583 -11.92793, -77.08598 -11.9..."
4,5,RE-1046,150125002000480,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Plaza Jerusalén,4727.502509,...,150125,0001,LIMA,LIMA,PUENTE PIEDRA,PUENTE PIEDRA,1501250001,4.593910e-07,0.002747,"POLYGON ((-77.11757 -11.82812, -77.11778 -11.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9405,4211,D-4150,150128022000420,NaN,NaN,NaN,OU:DEPORTE,COMPLEJO DEPORTIVO,RIMAC,771.507460,...,150128,0001,LIMA,LIMA,RIMAC,RIMAC,1501280001,5.585861e-07,0.002990,"POLYGON ((-77.03137 -12.04144, -77.0306 -12.04..."
9406,4212,D-1021,15012301201039B,NaN,NaN,NaN,OU:DEPORTE,CAMPO DEPORTIVO,VISTA ALEGRE,2117.492086,...,150123,0001,LIMA,LIMA,PACHACAMAC,PACHACAMAC,1501230001,1.683302e-07,0.001679,"POLYGON ((-76.88069 -12.09506, -76.88074 -12.0..."
9407,4213,D-1022,150133029000240,NaN,NaN,NaN,OU:DEPORTE,COMPLEJO DEPORTIVO,IPD SAN JUAN DE MIRAFLORES,54665.688321,...,150133,0001,LIMA,LIMA,SAN JUAN DE MIRAFLORES,CIUDAD DE DIOS,1501330001,1.176255e-05,0.014409,"POLYGON ((-76.97324 -12.1596, -76.9732 -12.158..."
9408,4214,D-1023,150143007000090,NaN,NaN,NaN,OU:DEPORTE,CAMPO DEPORTIVO,NaN,9301.308144,...,150143,0001,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,VILLA MARIA DEL TRIUNFO,1501430001,5.304972e-06,0.010955,"POLYGON ((-76.94306 -12.13771, -76.94314 -12.1..."


In [32]:
# Eliminar filas con valores duplicados en la columna ID_MANZANA
gdf_combined = gdf_combined.drop_duplicates(subset='ID_MANZANA', keep='first')

In [33]:
print("Se tienen ",gdf_combined.shape[0]," albergues potenciales entre centros recreativos y deportivos")

Se tienen  7420  albergues potenciales entre centros recreativos y deportivos


In [34]:
# Descargar el archivo de los albergues potenciales como SHAPEFILE
gdf_combined = gpd.GeoDataFrame(gdf_combined, geometry='geometry')

In [189]:
# Renombrar columnas largas a nombres más cortos (máximo 10 caracteres)
gdf_combined = gdf_combined.rename(columns={
    'Shape_Length': 'Shape_Len',
    # Agrega más renombramientos si es necesario
})

# Exportar a shapefile
gdf_combined.to_file('albergues_potenciales_lima.shp', driver='ESRI Shapefile')

### ELiminación de columnas sin detalle

In [35]:
# Eliminar múltiples columnas que no guardan información relevante
gdf_combined = gdf_combined.drop(columns=['OBJECTID', 'COD_EQ', 'IDLOTE', 'IDMZNAR', 'COD_SECT', 'AREA_M2', 'IDMANZANA', 'DEPARTAMEN', 'PROVINCIA', 'DISTRITO', 'NOMCCCPP', 'IDCCPP'])

In [36]:
gdf_combined.columns

Index(['ID_MANZANA', 'TIPO_EQ', 'SUBTIPO_EQ', 'NOM_EQ', 'OBSERV', 'NOMBDIST',
       'NOMBPROV', 'NOMBDEP', 'Shape_Area', 'Shape_Length', 'CODDPTO',
       'CODPROV', 'CODDIST', 'CODZONA', 'SUFZONA', 'CODMZNA', 'SUFMZNA',
       'UBIGEO', 'CODCCPP', 'Shape_STAr', 'Shape_STLe', 'geometry'],
      dtype='object')

In [37]:
gdf_combined.head()

,ID_MANZANA,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,OBSERV,NOMBDIST,NOMBPROV,NOMBDEP,Shape_Area,Shape_Length,...,CODDIST,CODZONA,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,geometry
0,150139002000130,RECREACION PUBLICA,PLAZA,Plaza Miami,CEMENTO,SANTA ROSA,LIMA,LIMA,8521.91,420.40,...,39,002,00,013,0,150139,0001,6.997925e-07,0.003825,"POLYGON ((-77.17481 -11.79968, -77.17481 -11.7..."
1,150139004000190,RECREACION PUBLICA,PLAZA PRINCIPAL,Plaza de Armas de Santa Rosa,CEMENTO,SANTA ROSA,LIMA,LIMA,3521.63,257.03,...,39,004,00,019,0,150139,0001,4.455260e-07,0.003180,"POLYGON ((-77.15797 -11.78584, -77.15771 -11.7..."
2,150125010000490,RECREACION PUBLICA,PARQUE LOCAL,NaN,VERDE,PUENTE PIEDRA,LIMA,LIMA,2521.86,397.11,...,25,010,00,049,0,150125,0001,3.431174e-07,0.003262,"POLYGON ((-77.11043 -11.83321, -77.11045 -11.8..."
3,15012504800011D,RECREACION PUBLICA,PLAZA,Plaza Virgen de la Medalla Milagrosa,VERDE,PUENTE PIEDRA,LIMA,LIMA,5417.73,329.34,...,25,048,00,011,D,150125,0001,5.113287e-07,0.002939,"POLYGON ((-77.08583 -11.92793, -77.08598 -11.9..."
4,150125002000480,RECREACION PUBLICA,PARQUE LOCAL,Plaza Jerusalén,VERDE,PUENTE PIEDRA,LIMA,LIMA,4727.50,275.62,...,25,002,00,048,0,150125,0001,4.593910e-07,0.002747,"POLYGON ((-77.11757 -11.82812, -77.11778 -11.8..."


### Agregación de Longitud y Latitud

In [38]:
gdf_combined_coor = gdf_combined.copy()

# Calcular los centroides si las geometrías no son puntos
gdf_combined_coor['centroid'] = gdf_combined_coor.geometry.centroid

# Extraer latitud y longitud desde los centroides
gdf_combined_coor['latitud'] = gdf_combined_coor['centroid'].y
gdf_combined_coor['longitud'] = gdf_combined_coor['centroid'].x


C:\Users\amyan\AppData\Local\Temp\ipykernel_35356\1442919720.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_combined_coor['centroid'] = gdf_combined_coor.geometry.centroid


In [ ]:
gdf_combined_coor

# 4. Caracterización de Albergues Potenciales (Con su misma manzana)

## Agregación de características 

In [40]:
df_manzanas_carac = pd.read_excel('BD_ManzanasCarac.xlsx')

In [41]:
df_manzanas_carac.shape

(106991, 13)

In [42]:
df_manzanas_carac.head()

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NOMBDIST_VULN,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,COB_AP17,COB_EE17,TIPO_EE17
0,150101001000010,BAJA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,164.0,170.0,0.065346,0.065346,CON COBERTURA,CON COBERTURA,1.0
1,150101001000020,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,4.0,4.0,0.001985,0.001985,CON COBERTURA,CON COBERTURA,1.0
2,150101001000030,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,56.0,58.0,0.005767,0.005767,CON COBERTURA,CON COBERTURA,1.0
3,150101001000040,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,235.0,244.0,0.043450,0.043450,CON COBERTURA,CON COBERTURA,1.0
4,150101001000050,ALTA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,361.0,374.0,0.071250,0.071250,CON COBERTURA,CON COBERTURA,1.0


In [43]:
# Cambiar el nombre de cobertura de Agua y Electricidad
df_manzanas_carac = df_manzanas_carac.rename(columns={'COB_AP17': 'COB_AGUA','COB_EE17': 'COB_ELEC'})

In [44]:
# Convertir la columna ID_MANZANA a string
df_manzanas_carac['ID_MANZANA'] = df_manzanas_carac['ID_MANZANA'].astype(str)

In [45]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre df_manzanas_carac y gdf_combined_coor
# La clave de unión es "ID_MANZANA"
gdf_albergues_carac = df_manzanas_carac.merge(gdf_combined_coor, left_on='ID_MANZANA', right_on='ID_MANZANA', how='inner')

In [46]:
print('Albergues potenciales caracterizados: ', gdf_albergues_carac.shape[0])

Albergues potenciales caracterizados:  7420


In [47]:
gdf_albergues_carac.head(3)

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NOMBDIST_VULN,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,...,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,geometry,centroid,latitud,longitud
0,150101001000020,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,4.0,4.0,0.001985,0.001985,...,002,0,150101,0001,1.753006e-07,0.001879,"POLYGON ((-77.08217 -12.03634, -77.0821 -12.03...",POINT (-77.08182 -12.03641),-12.036411,-77.081824
1,150101001000060,BAJA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,0.0,0.0,0.000000,0.000000,...,006,0,150101,0001,4.451045e-07,0.002856,"POLYGON ((-77.07984 -12.03645, -77.0798 -12.03...",POINT (-77.08031 -12.03674),-12.036744,-77.080308
2,150101001000200,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,132.0,137.0,0.016823,0.016823,...,020,0,150101,0001,6.615369e-07,0.003927,"POLYGON ((-77.07973 -12.03763, -77.07969 -12.0...",POINT (-77.08021 -12.03807),-12.038073,-77.080213


In [48]:
gdf_albergues_carac['GEOMETRY'] = gdf_albergues_carac.geometry
# Eliminar múltiples columnas que no guardan información relevante
gdf_albergues_carac = gdf_albergues_carac.drop(columns=['geometry'])

In [49]:
# Checar las columnas con nombre de mas de 10 caracteres
for col in list(gdf_albergues_carac.columns):
    if len(col)>10:
        print(col,len(col))

NOMBDIST_VULN 13
Shape_Length 12


In [50]:
# La columna NOMBDIST_VULN no tiene importancia, solo tiene un dato: 'LIMA', se procede a ELIMINAR
gdf_albergues_carac = gdf_albergues_carac.drop(columns=['NOMBDIST_VULN'])

In [51]:
# Eliminar la columna 'centroid' ya que no puede haber 2 columnas de tipo geometry
gdf_albergues_carac = gdf_albergues_carac.drop(columns=['centroid'])

In [52]:
# Descargar el archivo de los albergues potenciales como SHAPEFILE
gdf_albergues_carac = gpd.GeoDataFrame(gdf_albergues_carac, geometry='GEOMETRY')

In [272]:
# Exportar a shapefile
gdf_albergues_carac.to_file('albergues_caracterizados_lima.shp', driver='ESRI Shapefile')

In [53]:
gdf_albergues_carac

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,COB_AGUA,...,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,latitud,longitud,GEOMETRY
0,150101001000020,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,4.0,4.0,0.001985,0.001985,CON COBERTURA,...,00,002,0,150101,0001,1.753006e-07,0.001879,-12.036411,-77.081824,"POLYGON ((-77.08217 -12.03634, -77.0821 -12.03..."
1,150101001000060,BAJA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,0.0,0.0,0.000000,0.000000,NO DATA,...,00,006,0,150101,0001,4.451045e-07,0.002856,-12.036744,-77.080308,"POLYGON ((-77.07984 -12.03645, -77.0798 -12.03..."
2,150101001000200,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,132.0,137.0,0.016823,0.016823,CON COBERTURA,...,00,020,0,150101,0001,6.615369e-07,0.003927,-12.038073,-77.080213,"POLYGON ((-77.07973 -12.03763, -77.07969 -12.0..."
3,150101002000110,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,00,011,0,150101,0001,2.876209e-07,0.002247,-12.036321,-77.072672,"POLYGON ((-77.0723 -12.03609, -77.07232 -12.03..."
4,150101002000370,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,00,037,0,150101,0001,9.406034e-08,0.001231,-12.036579,-77.066697,"POLYGON ((-77.06684 -12.03675, -77.06684 -12.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7415,150143084000130,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,2.0,2.0,0.000753,0.000753,CON COBERTURA,...,00,013,0,150143,0001,2.303512e-07,0.001975,-12.231363,-76.908719,"POLYGON ((-76.90843 -12.23115, -76.90866 -12.2..."
7416,150143084000230,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,00,023,0,150143,0001,1.456864e-07,0.001527,-12.232505,-76.908669,"POLYGON ((-76.90843 -12.23238, -76.90855 -12.2..."
7417,150143084000250,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,00,025,0,150143,0001,1.182956e-07,0.001386,-12.231652,-76.908308,"POLYGON ((-76.90807 -12.23158, -76.90818 -12.2..."
7418,15014308400044A,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,00,044,A,150143,0001,4.669306e-08,0.000895,-12.230973,-76.905822,"POLYGON ((-76.90566 -12.231, -76.90577 -12.231..."


## Agrupación por Zonas Interdistritales: Albergues potenciales

In [71]:
from unidecode import unidecode

In [216]:
def custom_unidecode(text):
    text = text.upper()
    return ''.join([char if char=='Ñ' else unidecode(char) for char in text])

In [217]:
Distrito = [ "Ancón", "Carabayllo", "Comas", "Independencia", "Los Olivos", 
        "Puente Piedra", "San Martín de Porres", "Santa Rosa",
        "Ate","Chaclacayo" ,"Cieneguilla", "El Agustino","La Molina", 
        "Lurigancho", "San Juan de Lurigancho", "San Luis","Santa Anita",
        "Barranco", "Chorrillos", "Lurín", "Pachacámac", "Pucusana","Punta Hermosa", 
        "Punta Negra", "San Bartolo", "San Juan de Miraflores","Santa María del Mar", 
        "Villa El Salvador", "Villa María del Triunfo",
        "LIMA", "Breña", "Jesús María", "Lince", 
        "La Victoria", "Rímac", "San Borja",
        "Magdalena del Mar", "Miraflores", "Pueblo Libre", 
        "San Isidro", "Santiago de Surco","San Miguel", "Surquillo"]

Zona = ["Lima Norte", "Lima Norte", "Lima Norte", "Lima Norte", 
        "Lima Norte", "Lima Norte", "Lima Norte", "Lima Norte",
        "Lima Este", "Lima Este","Lima Este","Lima Este" , "Lima Este", 
        "Lima Este", "Lima Este", "Lima Este","Lima Este",
        "Lima Sur", "Lima Sur", "Lima Sur", "Lima Sur", 
        "Lima Sur", "Lima Sur", "Lima Sur", "Lima Sur", 
        "Lima Sur", "Lima Sur","Lima Sur","Lima Sur",
        "Lima Centro", "Lima Centro", "Lima Centro", "Lima Centro", 
        "Lima Centro", "Lima Centro", "Lima Centro",
        "Lima Centro", "Lima Centro", "Lima Centro", 
        "Lima Centro", "Lima Centro", "Lima Centro","Lima Centro"]

In [125]:
dist_uni = []
for dist in Distrito:
    dist_uni.append(custom_unidecode(dist))

In [127]:
zonifica = {i:j for i,j in zip(dist_uni,Zona)}

In [128]:
zonifica

{'ANCON': 'Lima Norte',
 'CARABAYLLO': 'Lima Norte',
 'COMAS': 'Lima Norte',
 'INDEPENDENCIA': 'Lima Norte',
 'LOS OLIVOS': 'Lima Norte',
 'PUENTE PIEDRA': 'Lima Norte',
 'SAN MARTIN DE PORRES': 'Lima Norte',
 'SANTA ROSA': 'Lima Norte',
 'ATE': 'Lima Este',
 'CHACLACAYO': 'Lima Este',
 'CIENEGUILLA': 'Lima Este',
 'EL AGUSTINO': 'Lima Este',
 'LA MOLINA': 'Lima Este',
 'LURIGANCHO': 'Lima Este',
 'SAN JUAN DE LURIGANCHO': 'Lima Este',
 'SAN LUIS': 'Lima Este',
 'SANTA ANITA': 'Lima Este',
 'BARRANCO': 'Lima Sur',
 'CHORRILLOS': 'Lima Sur',
 'LURIN': 'Lima Sur',
 'PACHACAMAC': 'Lima Sur',
 'PUCUSANA': 'Lima Sur',
 'PUNTA HERMOSA': 'Lima Sur',
 'PUNTA NEGRA': 'Lima Sur',
 'SAN BARTOLO': 'Lima Sur',
 'SAN JUAN DE MIRAFLORES': 'Lima Sur',
 'SANTA MARIA DEL MAR': 'Lima Sur',
 'VILLA EL SALVADOR': 'Lima Sur',
 'VILLA MARIA DEL TRIUNFO': 'Lima Sur',
 'LIMA': 'Lima Centro',
 'BREÑA': 'Lima Centro',
 'JESUS MARIA': 'Lima Centro',
 'LINCE': 'Lima Centro',
 'LA VICTORIA': 'Lima Centro',
 'RIMAC'

In [129]:
zonifica['SURQUILLO']='Lima Centro'

In [130]:
zonifica['SURQUILLO']

'Lima Centro'

In [131]:
def zoni(dist):
    return zonifica[dist]

In [133]:
gdf_albergues_carac['ZONA'] = gdf_albergues_carac['NOMBDIST'].apply(zoni)

In [137]:
zona_dist = gdf_albergues_carac.groupby('ZONA')

In [138]:
zona_dist.count()

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,COB_AGUA,...,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,latitud,longitud,GEOMETRY
ZONA,,,,,,,,,,,,,,,,,,,,,
Lima Centro,1314,1314,1314,1314,1314,1314,1314,1314,1314,1314,...,1314,1314,1314,1314,1314,1314,1314,1314,1314,1314
Lima Este,2153,2153,2153,2153,2153,2153,2153,2153,2153,2153,...,2153,2153,2153,2153,2153,2153,2153,2153,2153,2153
Lima Norte,2395,2395,2395,2395,2395,2395,2395,2395,2395,2395,...,2395,2395,2395,2395,2395,2395,2395,2395,2395,2395
Lima Sur,1558,1558,1558,1558,1558,1557,1557,1557,1557,1558,...,1558,1558,1558,1558,1558,1558,1558,1558,1558,1558


## Agrupación por Zonas Interdistritales: Manzanas

In [58]:
gdf_Lima.shape

(107100, 18)

In [59]:
df_manzanas_carac.shape

(106991, 13)

In [62]:
# Convertir la columna ID_MANZANA a string
df_manzanas_carac['ID_MANZANA'] = df_manzanas_carac['ID_MANZANA'].astype(str)

In [63]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre df_manzanas_carac y gdf_combined_coor
# La clave de unión es "ID_MANZANA"
gdf_manzanas_carac = df_manzanas_carac.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='inner')

In [64]:
print('Manzanas de Lima caracterizados: ', gdf_manzanas_carac.shape[0])

Manzanas de Lima caracterizados:  106970


In [139]:
gdf_manzanas_carac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106970 entries, 0 to 106969
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   ID_MANZANA     106970 non-null  object  
 1   NIV_VULNE      106970 non-null  object  
 2   DESC_VULNE     106970 non-null  object  
 3   NOMBDIST_VULN  106970 non-null  object  
 4   NIV_RIESGO     106970 non-null  object  
 5   DESC_RIESG     106970 non-null  object  
 6   TOT_POB17      106873 non-null  float64 
 7   TOT_POB20      106873 non-null  float64 
 8   DENS_POB17     106873 non-null  float64 
 9   DENS_POB20     106873 non-null  float64 
 10  COB_AGUA       106966 non-null  object  
 11  COB_ELEC       106966 non-null  object  
 12  TIPO_EE17      96961 non-null   float64 
 13  IDMANZANA      106970 non-null  object  
 14  CODDPTO        106970 non-null  object  
 15  CODPROV        106970 non-null  object  
 16  CODDIST        106970 non-null  object  
 17  CODZONA   

In [140]:
gdf_manzanas_carac['ZONA'] = gdf_manzanas_carac['DISTRITO'].apply(zoni)

In [142]:
zona_mzn = gdf_manzanas_carac.groupby('ZONA')

In [143]:
zona_mzn.count()

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NOMBDIST_VULN,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
ZONA,,,,,,,,,,,,,,,,,,,,,
Lima Centro,12779,12779,12779,12779,12779,12779,12778,12778,12778,12778,...,12779,12779,12779,12779,12779,12779,12779,12779,12779,12779
Lima Este,37358,37358,37358,37358,37358,37358,37310,37310,37310,37310,...,37358,37358,37358,37358,37358,37358,37358,37358,37358,37358
Lima Norte,31520,31520,31520,31520,31520,31520,31502,31502,31502,31502,...,31520,31520,31520,31520,31520,31520,31520,31520,31520,31520
Lima Sur,25313,25313,25313,25313,25313,25313,25283,25283,25283,25283,...,25313,25313,25313,25313,25313,25313,25313,25313,25313,25313


# 6. Caracterización de Albergues Potenciales (Con las manzanas a su alrededor)

## Importar BD manzanas caracterizadas

In [162]:
gdf_mzn_carac = gpd.read_file('manzanas_caracterizadas_lima.shp') # Distritos del norte de Irlanda

In [163]:
gdf_mzn_carac.head(3)

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,COB_AGUA,COB_ELEC,UBIGEO,DEPARTAMEN,PROVINCIA,DISTRITO,Shape_STAr,Shape_STLe,latitud,longitud,geometry
0,150101001000010,BAJA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,164.0,170.0,0.065346,0.065346,CON COBERTURA,CON COBERTURA,150101,LIMA,LIMA,LIMA,2.680216e-07,0.002237,-12.036511,-77.082584,"POLYGON ((-77.08219 -12.03661, -77.08295 -12.0..."
1,150101001000020,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,4.0,4.0,0.001985,0.001985,CON COBERTURA,CON COBERTURA,150101,LIMA,LIMA,LIMA,1.753006e-07,0.001879,-12.036411,-77.081824,"POLYGON ((-77.08217 -12.03634, -77.0821 -12.03..."
2,150101001000030,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,56.0,58.0,0.005767,0.005767,CON COBERTURA,CON COBERTURA,150101,LIMA,LIMA,LIMA,1.171026e-06,0.007603,-12.035964,-77.079727,"POLYGON ((-77.08014 -12.03582, -77.07995 -12.0..."


## Importar BD Albergues caracterizados

In [164]:
gdf_albergues = gpd.read_file('albergues_caracterizados_lima.shp') # Distritos del norte de Irlanda

In [165]:
gdf_albergues = gdf_albergues.rename(columns={'NIV_VULNE': 'A_VULNE','NIV_RIESGO': 'A_RIESGO','TOT_POB17':'A_POB17','COB_AGUA':'A_COBAGUA','COB_ELEC':'A_COBELEC','Shape_Area':'AREA','Shape_Len':'PERIMETRO'})

In [166]:
gdf_albergues = gdf_albergues.drop(columns=['DESC_VULNE', 'DESC_RIESG','TOT_POB20','DENS_POB17','DENS_POB20','TIPO_EE17','CODDPTO','CODPROV','CODDIST','CODZONA','SUFZONA','CODMZNA','SUFMZNA','CODCCPP','Shape_STAr','Shape_STLe'])

In [167]:
gdf_albergues.columns

Index(['ID_MANZANA', 'A_VULNE', 'A_RIESGO', 'A_POB17', 'A_COBAGUA',
       'A_COBELEC', 'TIPO_EQ', 'SUBTIPO_EQ', 'NOM_EQ', 'OBSERV', 'NOMBDIST',
       'NOMBPROV', 'NOMBDEP', 'AREA', 'PERIMETRO', 'UBIGEO', 'latitud',
       'longitud', 'geometry'],
      dtype='object')

In [168]:
gdf_albergues.head(3)

,ID_MANZANA,A_VULNE,A_RIESGO,A_POB17,A_COBAGUA,A_COBELEC,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,OBSERV,NOMBDIST,NOMBPROV,NOMBDEP,AREA,PERIMETRO,UBIGEO,latitud,longitud,geometry
0,150101001000020,SIN VULNE,SIN RIESGO,4.0,CON COBERTURA,CON COBERTURA,RECREACION PUBLICA,PARQUE LOCAL,None,TIERRA,LIMA,LIMA,LIMA,2015.30,201.90,150101,-12.036411,-77.081824,"POLYGON ((-77.08217 -12.03634, -77.0821 -12.03..."
1,150101001000060,BAJA,MEDIO,0.0,NO DATA,NO DATA,RECREACION PUBLICA,PARQUE LOCAL,Parque Los Soldados del Perú,CEMENTO,LIMA,LIMA,LIMA,3811.58,252.13,150101,-12.036744,-77.080308,"POLYGON ((-77.07984 -12.03645, -77.0798 -12.03..."
2,150101001000200,MEDIA,MEDIO,132.0,CON COBERTURA,CON COBERTURA,OU:DEPORTE,CAMPO DEPORTIVO,SIMON BOLIVAR,None,LIMA,LIMA,LIMA,3596.78,255.07,150101,-12.038073,-77.080213,"POLYGON ((-77.07973 -12.03763, -77.07969 -12.0..."


## Filtro de Albergues potenciales

In [170]:
gdf_albergues.A_RIESGO.value_counts()

A_RIESGO
SIN RIESGO    4999
ALTO          1478
MEDIO          611
MUY ALTO       332
Name: count, dtype: int64

In [178]:
mascara = ((gdf_albergues['A_VULNE'] == 'BAJA') | (gdf_albergues['A_VULNE'] == 'MEDIA') | (gdf_albergues['A_VULNE'] == 'SIN VULNE')) & ((gdf_albergues['A_RIESGO'] == 'MEDIO') | (gdf_albergues['A_RIESGO'] == 'SIN RIESGO')) 

# Filtrar el GeoDataFrame usando la máscara
gdf_albergues_filtro = gdf_albergues[mascara].copy()
print("Los albergues que son de recreación pública y deportivo con vulnerabilidad baja son: ", gdf_albergues_filtro.shape[0])

Los albergues que son de recreación pública y deportivo con vulnerabilidad baja son:  5559


In [193]:
gdf_albergues_filtro_1 = gdf_albergues_filtro[['TIPO_EQ','SUBTIPO_EQ','NOM_EQ','NOMBDIST','latitud','longitud']].copy()

In [200]:
nombre = (gdf_albergues_filtro_1.TIPO_EQ.fillna('') + ' - ' + 
          gdf_albergues_filtro_1.SUBTIPO_EQ.fillna('') + ' - ' + 
          gdf_albergues_filtro_1.NOM_EQ.fillna(''))

In [206]:
# Insertar la columna en la primera posición
gdf_albergues_filtro_1.insert(0, 'Albergue', nombre)

In [208]:
gdf_albergues_filtro_1 = gdf_albergues_filtro_1.drop(['TIPO_EQ','SUBTIPO_EQ','NOM_EQ'],axis=1)

In [214]:
gdf_albergues_filtro_1 = gdf_albergues_filtro_1.rename(columns={'Albergue': 'ALBERGUE','NOMBDIST':'DISTRITO','latitud':'LATITUD','longitud':'LONGITUD'})

In [215]:
gdf_albergues_filtro_1

,ALBERGUE,DISTRITO,LATITUD,LONGITUD
0,RECREACION PUBLICA - PARQUE LOCAL -,LIMA,-12.036411,-77.081824
1,RECREACION PUBLICA - PARQUE LOCAL - Parque Los...,LIMA,-12.036744,-77.080308
2,OU:DEPORTE - CAMPO DEPORTIVO - SIMON BOLIVAR,LIMA,-12.038073,-77.080213
3,RECREACION PUBLICA - PARQUE LOCAL - Parque 9 d...,LIMA,-12.036321,-77.072672
4,RECREACION PUBLICA - PARQUE LOCAL - Parque Mir...,LIMA,-12.036579,-77.066697
...,...,...,...,...
7413,RECREACION PUBLICA - PARQUE LOCAL - Parque Unión,VILLA MARIA DEL TRIUNFO,-12.224424,-76.906827
7416,RECREACION PUBLICA - PARQUE LOCAL - Parque San...,VILLA MARIA DEL TRIUNFO,-12.232505,-76.908669
7417,RECREACION PUBLICA - PARQUE LOCAL -,VILLA MARIA DEL TRIUNFO,-12.231652,-76.908308
7418,OU:DEPORTE - LOSA MULTIDEPORTIVA -,VILLA MARIA DEL TRIUNFO,-12.230973,-76.905822


In [266]:
gdf_albergues_filtro_2 = gdf_albergues_filtro[['NOM_EQ','NOMBDIST','latitud','longitud']].copy()

In [267]:
gdf_albergues_filtro_2 = gdf_albergues_filtro_2.rename(columns={'NOM_EQ': 'ALBERGUE','NOMBDIST':'DISTRITO','latitud':'LATITUD','longitud':'LONGITUD'})

In [268]:
gdf_albergues_filtro_2

,ALBERGUE,DISTRITO,LATITUD,LONGITUD
0,None,LIMA,-12.036411,-77.081824
1,Parque Los Soldados del Perú,LIMA,-12.036744,-77.080308
2,SIMON BOLIVAR,LIMA,-12.038073,-77.080213
3,Parque 9 de Diciembre,LIMA,-12.036321,-77.072672
4,Parque Mirones,LIMA,-12.036579,-77.066697
...,...,...,...,...
7413,Parque Unión,VILLA MARIA DEL TRIUNFO,-12.224424,-76.906827
7416,Parque San Camilo,VILLA MARIA DEL TRIUNFO,-12.232505,-76.908669
7417,None,VILLA MARIA DEL TRIUNFO,-12.231652,-76.908308
7418,None,VILLA MARIA DEL TRIUNFO,-12.230973,-76.905822


## UNION de Albergues potenciales y Albergues de la Municipalidad

In [190]:
df_albergues_muni = pd.read_excel("ALBERGUES TEMPORALES MUNICIPALIDAD.xlsx")

In [219]:
df_albergues_muni.head()

,CÓDIGO,ALBERGUE,DISTRITO,DIRECCIÓN,LATITUD,LONGITUD
0,ANC-01,CS Villa Ancon,Ancón,"Cruce de la Av. Bolivar y la Av. Huallaga, Aso...",-11.732951,-77.145762
1,ANC-02,Parque Ciudad PYME,Ancón,"12 de Octubre Cdra 7, Ciudad PYME",-11.769322,-77.153816
2,ANC-03,Plaza Civica de Áncon,Ancón,"Av. Marañon y Av. 11 de Enero, Asoc. Popular L...",-11.733322,-77.141440
3,ANC-04,Estadio Municipal,Ancón,"Av. Acceso a Ancon y Malecon Las Colinas, AH V...",-11.774607,-77.167563
4,ANC-05,Parque San Pedro de Ancón,Ancón,"Calle Santa Rosa, Asoc. San Pedro de Ancon",-11.782347,-77.158699


In [257]:
Distrito = [ "Ancón", "Carabayllo", "Comas", "Independencia", "Los Olivos", 
        "Puente Piedra", "San Martín de Porres", "Santa Rosa",
        "Ate","Chaclacayo" ,"Cieneguilla", "El Agustino","La Molina", 
        "Lurigancho-Chosica", "San Juan de Lurigancho", "San Luis","Santa Anita",
        "Barranco", "Chorrillos", "Lurín", "Pachacamac", "Pucusana","Punta Hermosa", 
        "Punta Negra", "San Bartolo", "San Juan de Miraflores","Santa María del Mar", 
        "Villa el Salvador", "Villa María del Triunfo",
        "Cercado de Lima", "Breña", "Jesús María", "Lince", 
        "La Victoria", "Rimac", "San Borja",
        "Magdalena del Mar", "Miraflores", "Pueblo Libre", 
        "San Isidro", "Santiago de Surco","San Miguel", "Surquillo"]

In [258]:
dist_unicode = ['ANCON', 'CARABAYLLO', 'COMAS', 'INDEPENDENCIA', 'LOS OLIVOS', 'PUENTE PIEDRA', 'SAN MARTIN DE PORRES',
             'SANTA ROSA', 'ATE', 'CHACLACAYO', 'CIENEGUILLA', 'EL AGUSTINO', 'LA MOLINA', 'LURIGANCHO', 'SAN JUAN DE LURIGANCHO',
             'SAN LUIS', 'SANTA ANITA', 'BARRANCO', 'CHORRILLOS', 'LURIN', 'PACHACAMAC', 'PUCUSANA', 'PUNTA HERMOSA', 'PUNTA NEGRA',
             'SAN BARTOLO', 'SAN JUAN DE MIRAFLORES', 'SANTA MARIA DEL MAR', 'VILLA EL SALVADOR', 'VILLA MARIA DEL TRIUNFO',
             'LIMA', 'BREÑA', 'JESUS MARIA', 'LINCE', 'LA VICTORIA', 'RIMAC', 'SAN BORJA', 'MAGDALENA DEL MAR', 'MIRAFLORES', 'PUEBLO LIBRE',
             'SAN ISIDRO', 'SANTIAGO DE SURCO', 'SAN MIGUEL', 'SURQUILLO']

In [259]:
dist_mayusc = {i:j for i,j in zip(Distrito,dist_unicode)}

In [247]:
def d_mayusc(dist):
    return dist_mayusc[dist]

In [261]:
nuevos_dist = []
for d in df_albergues_muni.DISTRITO.tolist():
    nuevos_dist.append(d_mayusc(d))

In [262]:
df_albergues_muni = df_albergues_muni.drop(['CÓDIGO','DISTRITO','DIRECCIÓN'],axis=1)

In [269]:
# Insertar la columna en la primera posición
df_albergues_muni.insert(1, 'DISTRITO', nuevos_dist)

In [285]:
df_albergues_muni.shape

(349, 4)

In [270]:
df_albergues_muni

,ALBERGUE,DISTRITO,LATITUD,LONGITUD
0,CS Villa Ancon,ANCON,-11.732951,-77.145762
1,Parque Ciudad PYME,ANCON,-11.769322,-77.153816
2,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440
3,Estadio Municipal,ANCON,-11.774607,-77.167563
4,Parque San Pedro de Ancón,ANCON,-11.782347,-77.158699
...,...,...,...,...
344,Complejo Deportivo Víctor Raúl Haya de La Torre,VILLA MARIA DEL TRIUNFO,-12.174541,-76.916667
345,Estadio Santa Rosa Las Conchitas,VILLA MARIA DEL TRIUNFO,-12.209796,-76.924180
346,Estadio Tablada 3er Sector,VILLA MARIA DEL TRIUNFO,-12.184128,-76.925897
347,Asociación Deportiva San Francisco 1er Sector,VILLA MARIA DEL TRIUNFO,-12.202415,-76.928861


In [354]:
# Hacemos el merge con indicador para saber de dónde proviene cada fila
albergues_merged = pd.merge(gdf_albergues_filtro_2, df_albergues_muni, on=['ALBERGUE', 'DISTRITO'], how='outer', indicator=True)

right = albergues_merged[albergues_merged['_merge'] == 'right_only']
both = albergues_merged[albergues_merged['_merge'] == 'both']
left = albergues_merged[albergues_merged['_merge'] == 'left_only']

In [355]:
# Albergues de la Municipalidad sin match
right = right.drop(['LATITUD_x','LONGITUD_x','_merge'],axis=1)
right = right.rename(columns={'LATITUD_y':'LATITUD','LONGITUD_y':'LONGITUD'})
right.head(3)

,ALBERGUE,DISTRITO,LATITUD,LONGITUD
5559,CS Villa Ancon,ANCON,-11.732951,-77.145762
5560,Parque Ciudad PYME,ANCON,-11.769322,-77.153816
5561,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440


In [356]:
right.shape

(296, 4)

In [357]:
# Albergues CON MATCH
both = both.drop(['LATITUD_y','LONGITUD_y','_merge'],axis=1)
both = both.rename(columns={'LATITUD_x':'LATITUD','LONGITUD_x':'LONGITUD'})
both.head(3)

,ALBERGUE,DISTRITO,LATITUD,LONGITUD
74,Parque Carlos Lisson,LIMA,-12.063444,-77.076901
75,Parque Santa Teodosia,LIMA,-12.062990,-77.074340
104,Parque Trinidad,LIMA,-12.051421,-77.065086


In [358]:
both.shape

(60, 4)

In [359]:
# Albergues potenciales sin match
left = left.drop(['LATITUD_y','LONGITUD_y','_merge'],axis=1)
left = left.rename(columns={'LATITUD_x':'LATITUD','LONGITUD_x':'LONGITUD'})
left.head(3)

,ALBERGUE,DISTRITO,LATITUD,LONGITUD
0,None,LIMA,-12.036411,-77.081824
1,None,LIMA,-12.036042,-77.065939
2,None,LIMA,-12.036117,-77.060637


In [360]:
# Hacemos el merge con indicador para saber de dónde proviene cada fila
albergue_nombre_left = pd.merge(gdf_albergues_filtro_1, left, on=['LATITUD', 'LONGITUD'], how='inner')

In [361]:
# Albergues potenciales sin match
albergue_nombre_left = albergue_nombre_left.drop(['ALBERGUE_y','DISTRITO_y'],axis=1)
albergue_nombre_left = albergue_nombre_left.rename(columns={'ALBERGUE_x':'ALBERGUE','DISTRITO_x':'DISTRITO'})
albergue_nombre_left.head(3)

,ALBERGUE,DISTRITO,LATITUD,LONGITUD
0,RECREACION PUBLICA - PARQUE LOCAL -,LIMA,-12.036411,-77.081824
1,RECREACION PUBLICA - PARQUE LOCAL - Parque Los...,LIMA,-12.036744,-77.080308
2,OU:DEPORTE - CAMPO DEPORTIVO - SIMON BOLIVAR,LIMA,-12.038073,-77.080213


In [362]:
albergue_nombre_left.shape

(5499, 4)

In [363]:
# Concatenar los DataFrames a nivel de filas
albergue_completo = pd.concat([right, both, albergue_nombre_left], axis=0)

In [369]:
# Generar la lista de valores desde A_0001 hasta A_5855
valores = [f"A_{str(i).zfill(4)}" for i in range(1, 5856)]

In [370]:
# Insertar la columna en la primera posición
albergue_completo.insert(0, 'ID_ALBERGUE', valores)

In [371]:
albergue_completo

,ID_ALBERGUE,ALBERGUE,DISTRITO,LATITUD,LONGITUD
5559,A_0001,CS Villa Ancon,ANCON,-11.732951,-77.145762
5560,A_0002,Parque Ciudad PYME,ANCON,-11.769322,-77.153816
5561,A_0003,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440
5562,A_0004,Estadio Municipal,ANCON,-11.774607,-77.167563
5563,A_0005,Parque San Pedro de Ancón,ANCON,-11.782347,-77.158699
...,...,...,...,...,...
5494,A_5851,RECREACION PUBLICA - PARQUE LOCAL - Parque Unión,VILLA MARIA DEL TRIUNFO,-12.224424,-76.906827
5495,A_5852,RECREACION PUBLICA - PARQUE LOCAL - Parque San...,VILLA MARIA DEL TRIUNFO,-12.232505,-76.908669
5496,A_5853,RECREACION PUBLICA - PARQUE LOCAL -,VILLA MARIA DEL TRIUNFO,-12.231652,-76.908308
5497,A_5854,OU:DEPORTE - LOSA MULTIDEPORTIVA -,VILLA MARIA DEL TRIUNFO,-12.230973,-76.905822


## Caracterización por manzanas: Vulnerabilidad sismica

In [110]:
import math

def calcular_rango(lat_central, lon_central, radio_km=1.5):
    # Constantes
    radio_tierra_km = 6371.01  # Radio de la Tierra en kilómetros
    km_por_grado_latitud = 111.32  # Aproximación de km por grado de latitud
    
    # Convertir radio de km a grados de latitud
    delta_latitud = radio_km / km_por_grado_latitud
    
    # Convertir radio de km a grados de longitud (dependiente de la latitud)
    delta_longitud = radio_km / (km_por_grado_latitud * math.cos(math.radians(lat_central)))
    
    # Rango de latitud y longitud
    lat_min = lat_central - delta_latitud
    lat_max = lat_central + delta_latitud
    lon_min = lon_central - delta_longitud
    lon_max = lon_central + delta_longitud
    
    return lat_min, lat_max, lon_min, lon_max

# Ejemplo de uso
lat_central = -12.0464  # Latitud de Lima, Perú
lon_central = -77.0428  # Longitud de Lima, Perú
rango = calcular_rango(lat_central, lon_central)
print(f"Rango de latitudes: {rango[0]:.6f} a {rango[1]:.6f}")
print(f"Rango de longitudes: {rango[2]:.6f} a {rango[3]:.6f}")


Rango de latitudes: -12.059875 a -12.032925
Rango de longitudes: -77.056578 a -77.029022


In [113]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from math import radians, sin, cos, sqrt, atan2 

def haversine(lon1, lat1, lon2, lat2):
    # Convertir grados a radianes
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Fórmula del semiverseno
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Radio de la Tierra en kilómetros
    r = 6371.01
    return c * r

In [375]:
def calcular_vulnerabilidad(gdf_mzn_carac, albergues_gdf, radio_km=1.5):
    
    for idx, albergue in albergues_gdf.iterrows():
        albergue_lat = albergue.LATITUD
        albergue_lon = albergue.LONGITUD
        
        # Calcular el rango de latitud y longitud
        lat_min, lat_max, lon_min, lon_max = calcular_rango(albergue_lat, albergue_lon, radio_km)

        # Aplicar la máscara
        mascara = (gdf_mzn_carac['latitud'] >= lat_min) & (gdf_mzn_carac['latitud'] <= lat_max) & \
                  (gdf_mzn_carac['longitud'] >= lon_min) & (gdf_mzn_carac['longitud'] <= lon_max)
        gdf_mzn_carac_mascara = gdf_mzn_carac[mascara].copy()
        # Seleccionar las manzanas con Vulnerabilidad Muy Alta para calcular su distancia
        gdf_mzn_dist = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['NIV_VULNE']=='MUY ALTA'].copy()

        # Calcular la distancia a todos las manzanas
        distancias = gdf_mzn_dist.apply(lambda manzana: haversine(albergue_lon, albergue_lat, manzana.longitud, manzana.latitud), axis=1)
        
        # Asignar la cantidad de manzanas
        albergues_gdf.at[idx, 'Manzanas'] = gdf_mzn_carac_mascara.shape[0]
        albergues_gdf.at[idx, 'M_VULNE_MUY_ALTA'] = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['NIV_VULNE']=='MUY ALTA'].shape[0]
        albergues_gdf.at[idx, 'M_VULNE_DIST'] = distancias.sum()
        #Promedio
        if distancias.empty:
            promedio = 0
        else:
            promedio = distancias.mean()
        albergues_gdf.at[idx, 'M_VULNE_DIST_PROM'] = promedio

    return albergues_gdf

In [376]:
albergues_vulne = albergue_completo.copy()
albergues_vulne = calcular_vulnerabilidad(gdf_mzn_carac,albergues_vulne,1)
albergues_vulne.head(3)

,ID_ALBERGUE,ALBERGUE,DISTRITO,LATITUD,LONGITUD,Manzanas,M_VULNE_MUY_ALTA,M_VULNE_DIST,M_VULNE_DIST_PROM
5559,A_0001,CS Villa Ancon,ANCON,-11.732951,-77.145762,349.0,5.0,3.187625,0.637525
5560,A_0002,Parque Ciudad PYME,ANCON,-11.769322,-77.153816,172.0,0.0,0.000000,0.000000
5561,A_0003,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440,304.0,3.0,2.701192,0.900397


## Caracterización por manzanas: Riesgo sísmico

In [182]:
gdf_mzn_carac.NIV_RIESGO.value_counts()

NIV_RIESGO
ALTO          53681
MUY ALTO      28828
MEDIO         16850
SIN RIESGO     7598
BAJO             13
Name: count, dtype: int64

In [377]:
def calcular_riesgo(gdf_mzn_carac, albergues_gdf, radio_km=1.5):
    
    for idx, albergue in albergues_gdf.iterrows():
        albergue_lat = albergue.LATITUD
        albergue_lon = albergue.LONGITUD
        
        # Calcular el rango de latitud y longitud
        lat_min, lat_max, lon_min, lon_max = calcular_rango(albergue_lat, albergue_lon, radio_km)

        # Aplicar la máscara
        mascara = (gdf_mzn_carac['latitud'] >= lat_min) & (gdf_mzn_carac['latitud'] <= lat_max) & \
                  (gdf_mzn_carac['longitud'] >= lon_min) & (gdf_mzn_carac['longitud'] <= lon_max)
        gdf_mzn_carac_mascara = gdf_mzn_carac[mascara].copy()
        # Seleccionar las manzanas con Vulnerabilidad Muy Alta para calcular su distancia
        gdf_mzn_dist = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['NIV_RIESGO']=='MUY ALTO'].copy()

        # Calcular la distancia a todos las manzanas
        distancias = gdf_mzn_dist.apply(lambda manzana: haversine(albergue_lon, albergue_lat, manzana.longitud, manzana.latitud), axis=1)
        
        # Asignar la cantidad de manzanas
        albergues_gdf.at[idx, 'M_RIESGO_MUY_ALTO'] = gdf_mzn_dist.shape[0]
        albergues_gdf.at[idx, 'M_RIESGO_DIST'] = distancias.sum()
        if distancias.empty:
            promedio = 0
        else:
            promedio = distancias.mean()
        albergues_gdf.at[idx, 'M_RIESGO_DIST_PROM'] = promedio

    return albergues_gdf

In [378]:
albergues_riesgo = albergues_vulne.copy()
albergues_riesgo = calcular_riesgo(gdf_mzn_carac,albergues_riesgo,1)
albergues_riesgo.head(3)

,ID_ALBERGUE,ALBERGUE,DISTRITO,LATITUD,LONGITUD,Manzanas,M_VULNE_MUY_ALTA,M_VULNE_DIST,M_VULNE_DIST_PROM,M_RIESGO_MUY_ALTO,M_RIESGO_DIST,M_RIESGO_DIST_PROM
5559,A_0001,CS Villa Ancon,ANCON,-11.732951,-77.145762,349.0,5.0,3.187625,0.637525,0.0,0.0,0.0
5560,A_0002,Parque Ciudad PYME,ANCON,-11.769322,-77.153816,172.0,0.0,0.000000,0.000000,0.0,0.0,0.0
5561,A_0003,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440,304.0,3.0,2.701192,0.900397,0.0,0.0,0.0


## Caracterización por manzanas: Población

In [187]:
gdf_mzn_carac.TOT_POB17.mean()

80.02260627099454

In [379]:
def calcular_poblacion(gdf_mzn_carac, albergues_gdf, radio_km=1.5):
    
    for idx, albergue in albergues_gdf.iterrows():
        albergue_lat = albergue.LATITUD
        albergue_lon = albergue.LONGITUD
        
        # Calcular el rango de latitud y longitud
        lat_min, lat_max, lon_min, lon_max = calcular_rango(albergue_lat, albergue_lon, radio_km)

        # Aplicar la máscara
        mascara = (gdf_mzn_carac['latitud'] >= lat_min) & (gdf_mzn_carac['latitud'] <= lat_max) & \
                  (gdf_mzn_carac['longitud'] >= lon_min) & (gdf_mzn_carac['longitud'] <= lon_max)
        gdf_mzn_carac_mascara = gdf_mzn_carac[mascara].copy()
        
        # Asignar la cantidad de manzanas
        albergues_gdf.at[idx, 'M_POB17'] = gdf_mzn_carac_mascara.TOT_POB17.sum()

    return albergues_gdf

In [380]:
albergues_pob = albergues_riesgo.copy()
albergues_pob = calcular_poblacion(gdf_mzn_carac,albergues_pob,1)
albergues_pob.head(3)

,ID_ALBERGUE,ALBERGUE,DISTRITO,LATITUD,LONGITUD,Manzanas,M_VULNE_MUY_ALTA,M_VULNE_DIST,M_VULNE_DIST_PROM,M_RIESGO_MUY_ALTO,M_RIESGO_DIST,M_RIESGO_DIST_PROM,M_POB17
5559,A_0001,CS Villa Ancon,ANCON,-11.732951,-77.145762,349.0,5.0,3.187625,0.637525,0.0,0.0,0.0,11118.0
5560,A_0002,Parque Ciudad PYME,ANCON,-11.769322,-77.153816,172.0,0.0,0.000000,0.000000,0.0,0.0,0.0,3812.0
5561,A_0003,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440,304.0,3.0,2.701192,0.900397,0.0,0.0,0.0,8679.0


## Caracterización por manzanas: Cobertura de Agua

In [381]:
gdf_mzn_carac.COB_AGUA.value_counts()

COB_AGUA
CON COBERTURA    64663
SIN COBERTURA    27906
NO DATA          14397
Name: count, dtype: int64

In [382]:
def calcular_cobertura_agua(gdf_mzn_carac, albergues_gdf, radio_km=1.5):
    
    for idx, albergue in albergues_gdf.iterrows():
        albergue_lat = albergue.LATITUD
        albergue_lon = albergue.LONGITUD
        
        # Calcular el rango de latitud y longitud
        lat_min, lat_max, lon_min, lon_max = calcular_rango(albergue_lat, albergue_lon, radio_km)

        # Aplicar la máscara
        mascara = (gdf_mzn_carac['latitud'] >= lat_min) & (gdf_mzn_carac['latitud'] <= lat_max) & \
                  (gdf_mzn_carac['longitud'] >= lon_min) & (gdf_mzn_carac['longitud'] <= lon_max)
        gdf_mzn_carac_mascara = gdf_mzn_carac[mascara].copy()
        
        # Asignar la cantidad de manzanas
        albergues_gdf.at[idx, 'M_COB_AGUA_CON'] = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['COB_AGUA']=='CON COBERTURA'].shape[0]
        albergues_gdf.at[idx, 'M_COB_AGUA_SIN'] = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['COB_AGUA']=='SIN COBERTURA'].shape[0]
        albergues_gdf.at[idx, 'M_COB_AGUA_NODATA'] = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['COB_AGUA']=='NO DATA'].shape[0]

    return albergues_gdf

In [384]:
albergues_agua = albergues_pob.copy()
albergues_agua = calcular_cobertura_agua(gdf_mzn_carac,albergues_agua,1)
albergues_agua.head(3)

,ID_ALBERGUE,ALBERGUE,DISTRITO,LATITUD,LONGITUD,Manzanas,M_VULNE_MUY_ALTA,M_VULNE_DIST,M_VULNE_DIST_PROM,M_RIESGO_MUY_ALTO,M_RIESGO_DIST,M_RIESGO_DIST_PROM,M_POB17,M_COB_AGUA_CON,M_COB_AGUA_SIN,M_COB_AGUA_NODATA
5559,A_0001,CS Villa Ancon,ANCON,-11.732951,-77.145762,349.0,5.0,3.187625,0.637525,0.0,0.0,0.0,11118.0,0.0,321.0,28.0
5560,A_0002,Parque Ciudad PYME,ANCON,-11.769322,-77.153816,172.0,0.0,0.000000,0.000000,0.0,0.0,0.0,3812.0,25.0,101.0,46.0
5561,A_0003,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440,304.0,3.0,2.701192,0.900397,0.0,0.0,0.0,8679.0,0.0,261.0,43.0


## Caracterización por manzanas: Cobertura de Electricidad

In [385]:
gdf_mzn_carac.COB_ELEC.value_counts()

COB_ELEC
CON COBERTURA    82564
NO DATA          14397
SIN COBERTURA    10005
Name: count, dtype: int64

In [386]:
def calcular_cobertura_agua(gdf_mzn_carac, albergues_gdf, radio_km=1.5):
    
    for idx, albergue in albergues_gdf.iterrows():
        albergue_lat = albergue.LATITUD
        albergue_lon = albergue.LONGITUD
        
        # Calcular el rango de latitud y longitud
        lat_min, lat_max, lon_min, lon_max = calcular_rango(albergue_lat, albergue_lon, radio_km)

        # Aplicar la máscara
        mascara = (gdf_mzn_carac['latitud'] >= lat_min) & (gdf_mzn_carac['latitud'] <= lat_max) & \
                  (gdf_mzn_carac['longitud'] >= lon_min) & (gdf_mzn_carac['longitud'] <= lon_max)
        gdf_mzn_carac_mascara = gdf_mzn_carac[mascara].copy()
        
        # Asignar la cantidad de manzanas
        albergues_gdf.at[idx, 'M_COB_ELEC_CON'] = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['COB_ELEC']=='CON COBERTURA'].shape[0]
        albergues_gdf.at[idx, 'M_COB_ELEC_SIN'] = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['COB_ELEC']=='SIN COBERTURA'].shape[0]
        albergues_gdf.at[idx, 'M_COB_ELEC_NODATA'] = gdf_mzn_carac_mascara[gdf_mzn_carac_mascara['COB_ELEC']=='NO DATA'].shape[0]

    return albergues_gdf

In [387]:
albergues_elec = albergues_agua.copy()
albergues_elec = calcular_cobertura_agua(gdf_mzn_carac,albergues_elec,1)
albergues_elec.head(3)

,ID_ALBERGUE,ALBERGUE,DISTRITO,LATITUD,LONGITUD,Manzanas,M_VULNE_MUY_ALTA,M_VULNE_DIST,M_VULNE_DIST_PROM,M_RIESGO_MUY_ALTO,M_RIESGO_DIST,M_RIESGO_DIST_PROM,M_POB17,M_COB_AGUA_CON,M_COB_AGUA_SIN,M_COB_AGUA_NODATA,M_COB_ELEC_CON,M_COB_ELEC_SIN,M_COB_ELEC_NODATA
5559,A_0001,CS Villa Ancon,ANCON,-11.732951,-77.145762,349.0,5.0,3.187625,0.637525,0.0,0.0,0.0,11118.0,0.0,321.0,28.0,236.0,85.0,28.0
5560,A_0002,Parque Ciudad PYME,ANCON,-11.769322,-77.153816,172.0,0.0,0.000000,0.000000,0.0,0.0,0.0,3812.0,25.0,101.0,46.0,81.0,45.0,46.0
5561,A_0003,Plaza Civica de Áncon,ANCON,-11.733322,-77.141440,304.0,3.0,2.701192,0.900397,0.0,0.0,0.0,8679.0,0.0,261.0,43.0,199.0,62.0,43.0


# DESCARGA DE BD!

In [389]:
albergues_elec.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5855 entries, 5559 to 5498
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_ALBERGUE         5855 non-null   object 
 1   ALBERGUE            5855 non-null   object 
 2   DISTRITO            5855 non-null   object 
 3   LATITUD             5855 non-null   float64
 4   LONGITUD            5855 non-null   float64
 5   Manzanas            5855 non-null   float64
 6   M_VULNE_MUY_ALTA    5855 non-null   float64
 7   M_VULNE_DIST        5855 non-null   float64
 8   M_VULNE_DIST_PROM   5855 non-null   float64
 9   M_RIESGO_MUY_ALTO   5855 non-null   float64
 10  M_RIESGO_DIST       5855 non-null   float64
 11  M_RIESGO_DIST_PROM  5855 non-null   float64
 12  M_POB17             5855 non-null   float64
 13  M_COB_AGUA_CON      5855 non-null   float64
 14  M_COB_AGUA_SIN      5855 non-null   float64
 15  M_COB_AGUA_NODATA   5855 non-null   float64
 16  M_COB_EL

In [388]:
albergues_elec.to_excel('Albergues_completo_caracterizado.xlsx', index=False)

In [ ]:
# PARA TRANFORMAR LA LATITUD Y LONGITUD EN PUNTO
from shapely.geometry import Point
import geopandas as gpd

# Crear una nueva columna 'geometry' con puntos
gdf['geometry'] = gdf.apply(lambda row: Point(row['longitud'], row['latitud']), axis=1)

# Convertir el DataFrame a un GeoDataFrame (si no lo es ya)
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')

# Verificar que el GeoDataFrame tiene la columna 'geometry'
print(gdf.head())

## Caracterización por manzanas: Cercanía a Hospitales (Distancia de Semiverseno)

Los Centros de Salud se puede sacar los 3 mas cercanos y su promedio, tambien agregar su tipo para agregarlo como coeficiente

In [48]:
gdf_salud = gpd.read_file('centros_salud_caracterizados_lima.shp')

In [49]:
gdf_salud.head(3)

,ID_MANZANA,INSTI_ES,NIV_AT_ES,GEST_ES,CAT_ES,NCAT_ES,NOM_EQ,POBATEN_20,AREA_M2,NIV_VULNE,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,Shape_STAr,Shape_STLe,latitud,longitud,geometry
0,150137013000570,ESSALUD,SEGUNDO NIVEL,PUBLICO,HOSPITAL I,II-1,HOSPITAL I JORGE VOTO BERNALES CORPANCHO,30000,103659.611494,ALTA,...,150137,0001,LIMA,LIMA,SANTA ANITA,0.00003,0.027115,-12.044022,-76.948021,"POLYGON ((-76.94462 -12.04628, -76.94708 -12.0..."
1,150137013000570,ESSALUD,SEGUNDO NIVEL,PUBLICO,HOSPITAL II ESPECIALIZADO,II-E,ESSALUD - CLINICA GERIATRICA SAN ISIDRO LABRADOR,30000,112.382560,ALTA,...,150137,0001,LIMA,LIMA,SANTA ANITA,0.00003,0.027115,-12.044022,-76.948021,"POLYGON ((-76.94462 -12.04628, -76.94708 -12.0..."
2,150137013000570,MINSA,TERCER NIVEL,PUBLICO,HOSPITAL III,III-1,HOSPITAL HERMILIO VALDIZAN,40000,46259.616205,ALTA,...,150137,0001,LIMA,LIMA,SANTA ANITA,0.00003,0.027115,-12.044022,-76.948021,"POLYGON ((-76.94462 -12.04628, -76.94708 -12.0..."


In [59]:
gdf_albergues = gpd.read_file('albergues_caracterizados_lima.shp')

In [60]:
gdf_albergues.head(3)

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,COB_AGUA,...,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,latitud,longitud,geometry
0,150101001000020,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,4.0,4.0,0.001985,0.001985,CON COBERTURA,...,00,002,0,150101,0001,1.753006e-07,0.001879,-12.036411,-77.081824,"POLYGON ((-77.08217 -12.03634, -77.0821 -12.03..."
1,150101001000060,BAJA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,0.0,0.0,0.000000,0.000000,NO DATA,...,00,006,0,150101,0001,4.451045e-07,0.002856,-12.036744,-77.080308,"POLYGON ((-77.07984 -12.03645, -77.0798 -12.03..."
2,150101001000200,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,132.0,137.0,0.016823,0.016823,CON COBERTURA,...,00,020,0,150101,0001,6.615369e-07,0.003927,-12.038073,-77.080213,"POLYGON ((-77.07973 -12.03763, -77.07969 -12.0..."


In [11]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from math import radians, sin, cos, sqrt, atan2

In [12]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from math import radians, sin, cos, sqrt, atan2 

def haversine(lon1, lat1, lon2, lat2):
    # Convertir grados a radianes
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Fórmula del semiverseno
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Radio de la Tierra en kilómetros
    r = 6371.01
    return c * r

In [57]:
def calcular_distancia_hospital_mas_cercano(albergues_gdf, hospitales_gdf):
    # Crear una nueva columna 'Dist_Hos' para almacenar la distancia al hospital más cercano
    albergues_gdf['Dist_Hos'] = np.nan

    for idx, albergue in albergues_gdf.iterrows():
        albergue_lat = albergue.latitud
        albergue_lon = albergue.longitud
        
        # Calcular la distancia a todos los hospitales
        distancias = hospitales_gdf.apply(
            lambda hospital: haversine(albergue_lon, albergue_lat, hospital.longitud, hospital.latitud), 
            axis=1
        )
        
        # Asignar la menor distancia a la columna 'Dist_Hos'
        albergues_gdf.at[idx, 'Dist_Hos'] = distancias.min()

    return albergues_gdf


In [21]:
gdf_albergues_hosp[gdf_albergues_hosp.Dist_Hos>1.5]

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,COB_AGUA,...,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,latitud,longitud,geometry,Dist_Hos
166,150102001010190,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,4.0,4.0,0.000864,0.000864,SIN COBERTURA,...,019,0,150102,0001,3.881161e-07,0.003028,-11.762957,-77.160000,"POLYGON ((-77.16001 -11.76255, -77.15936 -11.7...",1.719010
167,150102001020010,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,436.0,452.0,0.001052,0.001052,NO DATA,...,001,0,150102,0001,3.455082e-05,0.028880,-11.744896,-77.167100,"POLYGON ((-77.16923 -11.74844, -77.16907 -11.7...",2.655160
168,150102001020050,ALTA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,38.0,39.0,0.000348,0.000348,CON COBERTURA,...,005,0,150102,0001,7.291879e-06,0.012691,-11.756248,-77.170212,"POLYGON ((-77.17088 -11.75392, -77.16934 -11.7...",2.009908
235,150102009000210,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,021,0,150102,0001,1.279461e-07,0.001787,-11.774962,-77.186679,"POLYGON ((-77.18706 -11.77486, -77.18699 -11.7...",1.521847
237,15010201000055I,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,10.0,10.0,0.001527,0.001527,SIN COBERTURA,...,055,I,150102,0001,5.710218e-07,0.003133,-11.749596,-77.151313,"POLYGON ((-77.15077 -11.74946, -77.15091 -11.7...",1.922575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7288,15014304800001I,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,001,I,150143,0001,1.899798e-08,0.000565,-12.153551,-76.909077,"POLYGON ((-76.90914 -12.15353, -76.90913 -12.1...",1.888665
7289,150143048030380,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,038,0,150143,0001,3.849311e-07,0.002520,-12.150386,-76.893010,"POLYGON ((-76.89296 -12.15084, -76.89341 -12.1...",3.398698
7290,150143048030490,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,049,0,150143,0001,4.504076e-07,0.002712,-12.151710,-76.890639,"POLYGON ((-76.89056 -12.15219, -76.89108 -12.1...",3.432615
7291,150143048040170,SIN VULNE,SIN VULNERABILIDAD,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NO DATA,...,017,0,150143,0001,1.352373e-07,0.001486,-12.156218,-76.886976,"POLYGON ((-76.88713 -12.15643, -76.88723 -12.1...",3.530207
